In [1]:
import numpy as np
import pandas as pd
import time
from multiprocessing import Pool
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import axes3d, Axes3D
import random
%run utils.ipynb

In [2]:
#hit_id - 1 is the index of the hit
hit_idi = 0
ri = 1
phii = 2
zi = 3
volume_idi = 4
layer_idi = 5

d_phi = 3*np.pi/180
d_phi2 = 3*np.pi/180
d_z = 5
d_r = 5

z_max = 20#maximum distance along z axis of track origin

r_first_layer = 32.313498434074106
r_second_layer = 72.14554366164577
r_third_layer = 116.08917912535651
z_first_disk_right = 599.9792565549286
z_first_disk_left = -z_first_disk_right
z_second_disk_right = 699.9354521625164
z_second_disk_left = -z_second_disk_right

In [3]:
def trans_to_cylindrical (hits):
    hits_trans_info = []
    for row in hits.itertuples():
        hit_id = row.__getattribute__('hit_id')
        x = row.__getattribute__('x')
        y = row.__getattribute__('y')
        z = row.__getattribute__('z')
        volume_id = row.__getattribute__('volume_id')
        layer_id = row.__getattribute__('layer_id')
        r, phi, z = cartesian_to_3d_polar(x,y,z)
        hits_trans_info.append([hit_id, r, phi, z, volume_id, layer_id])
    hits_trans = pd.DataFrame (hits_trans_info, columns=["hit_id", "r", "phi", "z", "volume_id", "layer_id"])
    return hits_trans


In [4]:
def quadrant_shift (phi):
    if (phi < 0):
        return 2*np.pi + phi
    else:
        return phi

In [5]:
def extrapolate (x1, x2, x3, y1, y2):
    #assert (x2 - x1 != 0)
    m = (y2 - y1)/(x2 - x1)
    b = y1 - x1*m
    y3 = x3*m + b
    return y3

In [6]:
def create_seeds (hits):#already transformed to cylindrical
    
    hits_trans = trans_to_cylindrical (hits)
    hits_array = np.array (hits_trans.values)
    
    hits_first_layer = hits_trans.loc[(hits_trans['volume_id'] == 8) & (hits_trans['layer_id'] == 2)]
    hits_first_layer_array = hits_first_layer.get_values()
    hits_second_layer = hits_trans.loc[(hits_trans['volume_id'] == 8) & (hits_trans['layer_id'] == 4)]
    hits_second_layer_array = hits_second_layer.get_values()
    hits_third_layer = hits_trans.loc[(hits_trans['volume_id'] == 8) & (hits_trans['layer_id'] == 6)]
    hits_third_layer_array = hits_third_layer.get_values()
    hits_first_disk_left = hits_trans.loc[(hits_trans['volume_id'] == 7) & (hits_trans['layer_id'] == 14)]
    hits_first_disk_left_array = hits_first_disk_left.get_values()
    hits_first_disk_right = hits_trans.loc[(hits_trans['volume_id'] == 9) & (hits_trans['layer_id'] == 2)]
    hits_first_disk_right_array = hits_first_disk_right.get_values()
    hits_second_disk_left = hits_trans.loc[(hits_trans['volume_id'] == 7) & (hits_trans['layer_id'] == 12)]
    hits_second_disk_left_array = hits_second_disk_left.get_values()
    hits_second_disk_right = hits_trans.loc[(hits_trans['volume_id'] == 9) & (hits_trans['layer_id'] == 4)]
    hits_second_disk_right_array = hits_second_disk_right.get_values()
    
    seeds = []
    pool = Pool(32)
    parameters = []
    for hit in hits_first_layer_array:
        parameters.append((hits_array, hits_second_layer_array, hits_third_layer_array, hits_first_disk_left_array, hits_first_disk_right_array, hits_second_disk_left_array, hits_second_disk_right_array, int(np.round(hit[hit_idi]))))
    seeds = pool.starmap (create_seeds_from_first_hit, parameters)
    #for parameter in parameters:
        #create_seeds_from_first_hit (parameter[0], parameter[1], parameter[2], parameter[3], parameter[4], parameter[5], parameter[6], parameter[7])
    flattened_seeds = []
    for some_seeds in seeds:
        for seed in some_seeds:
            flattened_seeds.append (seed)
    return flattened_seeds

In [20]:
def create_seeds_from_first_hit (hits_array, hits_second_layer_array, hits_third_layer_array, hits_first_disk_left_array, hits_first_disk_right_array, hits_second_disk_left_array, hits_second_disk_right_array, hit_id):
    seeds = []
    hit = hits_array[hit_id - 1]
    assert (hit[layer_idi] == 2)
    phi = hit[phii]
    z = hit[zi]
    if (abs(phi) > np.pi/2):
        phi = quadrant_shift (phi)
    phiMin = phi - d_phi/2
    phiMax = phi + d_phi/2
    zMin = extrapolate (0, r_first_layer, r_second_layer, z_max, z)
    zMax = extrapolate (0, r_first_layer, r_second_layer, -z_max, z)
    for thisHit in hits_second_layer_array:
        thisPhi = thisHit[phii]
        if (phiMax > np.pi):
            thisPhi = quadrant_shift (thisPhi)
        thisZ = thisHit[zi]
        if (thisPhi < phiMax and thisPhi > phiMin and thisZ < zMax and thisZ > zMin):
            seeds += create_seeds_from_second_hit (hits_array, hits_third_layer_array, (hit_id, int(np.round(thisHit[hit_idi]))))
            seeds += create_seeds_from_second_hit_first_disk (hits_array, hits_first_disk_left_array, hits_first_disk_right_array, (hit_id, int(np.round(thisHit[hit_idi]))))
    
    r = hit[ri]
    if (z > z_max):
        rMax = extrapolate (z_max, z, z_first_disk_right, 0, r)
        rMin = extrapolate (-z_max, z, z_first_disk_right, 0, r)
        for thisHit in hits_first_disk_right_array:
            thisPhi = thisHit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = thisHit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                seeds += create_seeds_from_second_hit_second_disk (hits_array, hits_second_disk_left_array, hits_second_disk_right_array, (hit_id, int(np.round(thisHit[hit_idi]))))
    if (z < -z_max):
        rMin = extrapolate (z_max, z, z_first_disk_left, 0, r)
        rMax = extrapolate (-z_max, z, z_first_disk_left, 0, r)
        for thisHit in hits_first_disk_left_array:
            thisPhi = thisHit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = thisHit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                seeds += create_seeds_from_second_hit_second_disk (hits_array, hits_second_disk_left_array, hits_second_disk_right_array, (hit_id, int(np.round(thisHit[hit_idi]))))
    return seeds

In [8]:
def create_seeds_from_second_hit (hits_array, hits_third_layer_array, twoSeed):
    seeds = []
    phi1 = hits_array[twoSeed[0] - 1][phii]
    phi2 = hits_array[twoSeed[1] - 1][phii]
    r1 = hits_array[twoSeed[0] - 1][ri]
    r2 = hits_array[twoSeed[1] - 1][ri]
    r3 = r_third_layer
    z1 = hits_array[twoSeed[0] - 1][zi]
    z2 = hits_array[twoSeed[1] - 1][zi]
    phi3 = extrapolate (r1, r2, r3, phi1, phi2)
    z3 = extrapolate (r1, r2, r3, z1, z2)
    if (abs(phi3) > np.pi/2):
        phi3 = quadrant_shift (phi3)
    phiMin = phi3 - d_phi2/2
    phiMax = phi3 + d_phi2/2
    zMin = z3 - d_z/2
    zMax = z3 + d_z/2
    for hit in hits_third_layer_array:
        thisPhi = hit[phii]
        if (phiMax > np.pi):
            thisPhi = quadrant_shift (thisPhi)
        thisZ = hit[zi]
        if (thisPhi < phiMax and thisPhi > phiMin and thisZ < zMax and thisZ > zMin):
            seeds.append((twoSeed[0], twoSeed[1], int(np.round(hit[0]))))
    return seeds

In [9]:
def create_seeds_from_second_hit_first_disk (hits_array, hits_first_disk_left_array, hits_first_disk_right_array, twoSeed):
    seeds = []
    phi1 = hits_array[twoSeed[0] - 1][phii]
    phi2 = hits_array[twoSeed[1] - 1][phii]
    r1 = hits_array[twoSeed[0] - 1][ri]
    r2 = hits_array[twoSeed[1] - 1][ri]
    z1 = hits_array[twoSeed[0] - 1][zi]
    z2 = hits_array[twoSeed[1] - 1][zi]
    if (z2 > z1):
        z3 = z_first_disk_right
    else:
        z3 = z_first_disk_left
    phi3 = extrapolate (z1, z2, z3, phi1, phi2)
    r3 = extrapolate (z1, z2, z3, r1, r2)
    if (abs(phi3) > np.pi/2):
        phi3 = quadrant_shift (phi3)
    phiMin = phi3 - d_phi2/2
    phiMax = phi3 + d_phi2/2
    rMin = r3 - d_r/2
    rMax = r3 + d_r/2
    if (z2 > z1):
        for hit in hits_first_disk_right_array:
            thisPhi = hit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = hit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                    seeds.append((twoSeed[0], twoSeed[1], int(np.round(hit[0]))))
    else:
        for hit in hits_first_disk_left_array:
            thisPhi = hit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = hit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                    seeds.append((twoSeed[0], twoSeed[1], int(np.round(hit[0]))))
    return seeds

In [10]:
def create_seeds_from_second_hit_second_disk (hits_array, hits_second_disk_left_array, hits_second_disk_right_array, twoSeed):
    seeds = []
    phi1 = hits_array[twoSeed[0] - 1][phii]
    phi2 = hits_array[twoSeed[1] - 1][phii]
    r1 = hits_array[twoSeed[0] - 1][ri]
    r2 = hits_array[twoSeed[1] - 1][ri]
    z1 = hits_array[twoSeed[0] - 1][zi]
    z2 = hits_array[twoSeed[1] - 1][zi]
    if (z2 > z1):
        z3 = z_second_disk_right
    else:
        z3 = z_second_disk_left
    phi3 = extrapolate (z1, z2, z3, phi1, phi2)
    r3 = extrapolate (z1, z2, z3, r1, r2)
    if (abs(phi3) > np.pi/2):
        phi3 = quadrant_shift (phi3)
    phiMin = phi3 - d_phi2/2
    phiMax = phi3 + d_phi2/2
    rMin = r3 - d_r/2
    rMax = r3 + d_r/2
    if (z2 > z1):
        for hit in hits_second_disk_right_array:
            thisPhi = hit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = hit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                    seeds.append((twoSeed[0], twoSeed[1], int(np.round(hit[0]))))
    else:
        for hit in hits_second_disk_left_array:
            thisPhi = hit[phii]
            if (phiMax > np.pi):
                thisPhi = quadrant_shift (thisPhi)
            thisR = hit[ri]
            if (thisPhi < phiMax and thisPhi > phiMin and thisR < rMax and thisR > rMin):
                    seeds.append((twoSeed[0], twoSeed[1], int(np.round(hit[0]))))
    return seeds

In [21]:
#testing
#get hits dataframe
hits, cells, particles, truth = load_data_single_event(1050)

#transform to cylindrical
hits_trans = trans_to_cylindrical (hits)

In [22]:

seeds = create_seeds(hits)
print (len (seeds))
np.save ('./seeds/training_event_v2_1050.npy', np.array (seeds))


orig = np.load('./seeds/training_event_1050.npy')

/home/dl-one/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel/__main__.py:3: RuntimeWarning: divide by zero encountered in double_scalars
  app.launch_new_instance()


28380


In [15]:
orig.shape

(16517, 3)

In [12]:
def rpzlv_from_seeds (seeds, hits):
    hits_from_seeds = []
    for seed in seeds:
        hits_from_seed = []
        for hit_id in seed:
            hit = hits.loc[hits['hit_id'] == hit_id]
            x = hit['x'].values[0]
            y = hit['y'].values[0]
            z = hit['z'].values[0]
            r, phi, z = cartesian_to_3d_polar (x, y, z)
            l = hit['layer_id'].values[0]
            v = hit['volume_id'].values[0]
            hits_from_seed.append ([r, phi, z, l, v])
        hits_from_seeds.append (hits_from_seed)
    return np.array (hits_from_seeds)

In [14]:
itr = load_dataset ('./data/test_data', parts = ['hits'])
for event_data in itr:
    event_id = event_data[0]
    if (event_id < 106):
        continue
    print (event_id)
    hits = event_data[1]
    hits_trans = trans_to_cylindrical (hits)
    seeds = create_seeds (hits)
    np.save ('./seeds/test_event_' + str (event_id) + '_.npy', np.array (seeds))

106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124


In [15]:
#ax = Axes3D(plt.figure(figsize=(20,20)))
#for seed in xyz:
#    ax.scatter(seed[0], seed[1], seed[2])
#    ax.plot(seed[0], seed[1], seed[2])
#ax.view_init(0,0)
#ax.set_xlabel('r (mm)')
#ax.set_ylabel('phi (radians)')
#ax.set_zlabel('z (mm)')
#plt.show()